In [1]:
import os
import numpy as np
import json
import torch
from collections import defaultdict

device = torch.device('cuda:1')

# data_type = 'train_5k'
# data_type = 'test_5k'
data_type = 'test_5k_2nd'

data_path = '/home/tione/notebook/algo-2021/dataset/tagging/tagging_dataset_%s' % data_type

text_dir = os.path.join(data_path, 'text_txt/tagging')

In [2]:
from transformers import BertTokenizer, BertModel
# from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
txt_embedding = BertModel.from_pretrained('bert-base-chinese').to(device)
# tokenizer = BertTokenizer.from_pretrained('hfl/chinese-bert-wwm')
# txt_embedding = BertModel.from_pretrained('hfl/chinese-bert-wwm').to(device)

comet_ml is installed but `COMET_API_KEY` is not set.


Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# extract text feature

for idx, item in enumerate(os.listdir(text_dir)):
    if idx % 100 == 0:
        print(idx)
        
    item_name = item.split('.')[0]
    
#     if os.path.exists(os.path.join('./%s/text_feature_wwm' % data_type, item_name + '.npy')):
#         continue
    
    item_path = os.path.join(text_dir, item)
    
    with open(item_path, 'r') as file:
        text_content = json.load(file)
        
    asr_list = text_content["video_asr"].split('|')
    ocr_list = text_content["video_ocr"].split("|")
    
    asr_embedding_list = list()
    
    for text in asr_list:
        text_token = tokenizer(text[:512], return_tensors='pt', padding=True)
        
        for token_item in text_token:
            try:
                text_token[token_item] = text_token[token_item].to(device)
            except:
                pass
            
        text_embedding = txt_embedding(**text_token)['pooler_output']
        asr_embedding_list.append(text_embedding.detach().cpu().numpy())
    
    data = np.concatenate(asr_embedding_list, axis=0)
    np.save(os.path.join('./%s/asr_feature' % data_type, item.split('.')[0]), data)
    
    ocr_embedding_list = list()
    
    for text in ocr_list:
        text_token = tokenizer(text[:512], return_tensors='pt', padding=True)
        
        for token_item in text_token:
            try:
                text_token[token_item] = text_token[token_item].to(device)
            except:
                pass
            
        text_embedding = txt_embedding(**text_token)['pooler_output']
        ocr_embedding_list.append(text_embedding.detach().cpu().numpy())
    
    data = np.concatenate(ocr_embedding_list, axis=0)
    np.save(os.path.join('./%s/ocr_feature' % data_type, item.split('.')[0]), data)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500


In [34]:
def resplit_text(text_list, target_length):
    while True:
        new_text_list = list()
        idx = 0

        while True:
            if (idx + 1) < len(text_list):
                new_text_list.append(text_list[idx] + text_list[idx+1])
            else:
                new_text_list.append(text_list[idx])

            idx += 2

            if idx >= len(text_list):
                break

        if len(new_text_list) <= target_length:
            break
        else:
            text_list = new_text_list
    
    return new_text_list

In [ ]:
# extract text feature

for idx, item in enumerate(os.listdir(text_dir)):
    if idx % 100 == 0:
        print(idx)
        
    item_name = item.split('.')[0]
    
    if os.path.exists(os.path.join('./%s/text_feature_wwm' % data_type, item_name + '.npy')):
        continue
    
    item_path = os.path.join(text_dir, item)
    
    with open(item_path, 'r') as file:
        text_content = json.load(file)
        
    asr_list = text_content["video_asr"].split('|')
    ocr_list = text_content["video_ocr"].split("|")
    
    # 根据list长度 对文字进行重新切分和组合
    # 使得长度不超过35
    
    if len(asr_list) > 35:
        asr_list = resplit_text(asr_list, 35)
        
    if len(ocr_list) > 35:
        ocr_list = resplit_text(ocr_list, 35)
    
    text_list = asr_list + ocr_list
    
    embedding_list = list()
    
    for text in text_list:
        text_token = tokenizer(text[:512], return_tensors='pt', padding=True)
        
        for token_item in text_token:
            try:
                text_token[token_item] = text_token[token_item].to(device)
            except:
                pass
            
        text_embedding = txt_embedding(**text_token)['pooler_output']
        embedding_list.append(text_embedding.detach().cpu().numpy())
    
    data = np.concatenate(embedding_list, axis=0)
    np.save(os.path.join('./%s/text_feature_wwm' % data_type, item.split('.')[0]), data)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
